<a href="https://colab.research.google.com/github/enVives/Caltech101/blob/main/Caltech101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
from itertools import filterfalse
import torch,torchvision
import numpy as np
import pandas as pd
import cv2
import time
import os

from torchvision import transforms,models
from torch import nn
from IPython.display import clear_output,display
from torch.utils.data import DataLoader, random_split,Subset,Dataset
from google.colab.patches import cv2_imshow
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from glob import glob

ROOT = './sample_data/'
CLASS_A = 'cougar_body'
CLASS_B = 'windsor_chair'

DOWNLOAD = False
#!rm -rf /content/sample_data/*
# torch.cuda.default_stream(torch.device('cuda'))

In [46]:
def find_mean_std(dataset):
  dataloader = DataLoader(dataset,batch_size = 1,shuffle=FALSE)
  mean = torch.zeros(3)
  std = torch.zeros(3)
  samples = 0

  for image,_ in dataloader:
    mean += image.mean(dim=[0, 2, 3])  # Mean for each channel [batchsize,channels,height,width]
    std += image.std(dim=[0, 2, 3])    # Std for each channel
    samples += 1

  mean /= samples
  std /= samples

  print(mean)
  print(std)

In [47]:
class Formes(Dataset):
    def __init__(self, paths, labels= None, transforms = None):
        self.images = paths
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.labels)

    def __getdist__(self):
      return pd.Series(self.labels).value_counts()

    def __getitem__(self, index):
      path = self.images[index]
      label = self.labels[index]

      image = cv2.imread(path, cv2.IMREAD_COLOR)
      image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
      image = self.transforms(image)

      return image, label

In [48]:
def get_labels(sett):
  labels = np.array([])
  frequency = np.zeros(101)
  for _,label in sett:
    labels = np.append(labels,label)
    frequency[label] += 1
  return labels.astype(int),frequency

In [49]:
#from pickle import FALSE
from sklearn.model_selection import train_test_split

TRAINING = 0.80
VAL = 0.10
TESTING = 0.10

#[0.5459, 0.5288, 0.5022]
#[0.2424, 0.2393, 0.2409]
mean = torch.tensor([0.5459, 0.5288, 0.5022])
std = torch.tensor([0.2424, 0.2393, 0.2409])

transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB") if img.mode != "RGB" else img),  # Ensure all images are RGB
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

dataset = torchvision.datasets.Caltech101(root= ROOT,download=DOWNLOAD,transform=transform)
img_files = sorted(glob('/content/sample_data/caltech101/101_ObjectCategories/*/*'))

labels = []
for img_path in img_files:
  label = img_path.split(os.path.sep)[-2]
  labels.append(label)

lb = LabelEncoder()
labels = lb.fit_transform(labels)

weights = 1.0 / pd.Series(labels).value_counts()
weights = weights / weights.sum()
weights = torch.tensor(weights,dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(img_files, labels, test_size=TESTING, random_state=42, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=VAL/(TRAINING+VAL), random_state=42, stratify=y_train)

training = Formes(X_train,y_train,transform)
testing = Formes(X_test,y_test,transform)
validation = Formes(X_val,y_val,transform)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#training,validation,testing = random_split(dataset,[train_size,validation_size,len(dataset)-train_size-validation_size])

In [50]:
print(f"LONGITUD SET DE TRAINING: {training.__len__()}")
print(f"Distribució training: {training.__getdist__()}")
print(f"LONGITUD SET DE VALIDATION: {validation.__len__()}")
print(f"Distribució validation: {validation.__getdist__()}")
print(f"LONGITUD SET DE TESTING: {testing.__len__()}")

LONGITUD SET DE TRAINING: 7315
Distribució training: 6     640
4     638
0     374
2     347
1     347
     ... 
84     28
44     28
12     27
65     26
53     25
Name: count, Length: 102, dtype: int64
LONGITUD SET DE VALIDATION: 915
Distribució validation: 4     80
6     80
0     47
1     44
2     44
      ..
45     3
74     3
98     3
65     3
53     3
Name: count, Length: 102, dtype: int64
LONGITUD SET DE TESTING: 915


Ara ja sabem que hem de predir si una imatge pertany a les classes 25 o 99

In [51]:
def pick_algorithm(number):
  if number == 0:
    alexnetmulticlass = models.alexnet(weights=None)

    alexnetmulticlass.classifier = nn.Sequential(
    torch.nn.Linear(9216, 1024),
    nn.ReLU(),
    torch.nn.Linear(1024, 1024),
    nn.ReLU(),
    torch.nn.Linear(1024, 512),
    nn.ReLU(),
    torch.nn.Linear(512, 102),  # Ja que tenim 101 classes.
    )
    loss_fn = nn.CrossEntropyLoss(weight=weights.to(device))
    #weight=class_weights.to(device)
    return alexnetmulticlass,loss_fn

In [52]:
def veure_imatges(train_data,std,mean):
  for i in range(len(train_data)):
    imatge,label = train_data[i]

    print(imatge.ndimension())
    print(imatge.shape)

    imatge = imatge * (std[:, None, None]*255) + (mean[:, None, None]*255)
    # Convert the tensor back to a NumPy array
    img_numpy = imatge.permute(1, 2, 0).numpy()  # Change from (C, H, W) to (H, W, C)
    cv2_imshow(img_numpy)
    time.sleep(5)
    clear_output(wait=True)

In [53]:
BATCH_SIZE = 64
EPOCHS = 95

train_loader = torch.utils.data.DataLoader(training, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation, batch_size=BATCH_SIZE, shuffle=True)
testing_loader = torch.utils.data.DataLoader(testing, batch_size=BATCH_SIZE, shuffle=True)

model,loss_fn = pick_algorithm(0)
model.to(device)

learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#veure_imatges(training,std,mean)

In [54]:
img, target = next(iter(train_loader))
print(img.shape)

torch.Size([64, 3, 224, 224])


In [55]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from tqdm.auto import tqdm
import pylab as pl

t_loss = np.zeros(EPOCHS)
v_loss = np.zeros(EPOCHS)
acc_t = np.zeros(EPOCHS) #accuracy
acc_v = np.zeros(EPOCHS)
f1_t = np.zeros(EPOCHS) #f1
f1_v = np.zeros(EPOCHS)
recall_t = np.zeros(EPOCHS) #recall
recall_v = np.zeros(EPOCHS)
precision_t = np.zeros(EPOCHS)
precision_v = np.zeros(EPOCHS) #precisió


pbar = tqdm(range(1, EPOCHS + 1))  # tdqm permet tenir text dinàmic

classes = []

def extreu_classes(target):
    for i in target:
      if i not in classes:
        classes.append(i)

for epoch in pbar:

    train_loss = 0
    train_acc = 0
    train_f1 = 0
    train_recall = 0
    train_precision = 0
    val_acc = 0
    val_recall = 0
    val_f1 = 0
    val_loss = 0
    val_precision = 0

    batch_num = 1

    for batch_num, (input_img, target) in tqdm(enumerate(train_loader), desc=f"Batches (Època {epoch})"):

        model.train()

        optimizer.zero_grad()
        #extreu_classes(target.to(device))

        output = model(input_img.to(device))

        # print(target.shape)
        # print(output.shape)

        target = target.to(device)
        loss = loss_fn(output, target)


        model.zero_grad()
        loss.backward()

        with torch.no_grad():
            optimizer.step()

        #print(f"Pèrdua entrenament batch: {batch_num} epoch: {epoch+1}  train_loss: {loss.item()}")
        model.eval()

        y_class_predict = torch.argmax(output, dim=1)
        y_class_predict = (y_class_predict.cpu().detach().numpy())
        target = target.cpu().detach().numpy()


        train_acc += accuracy_score(target,y_class_predict)
        train_f1 += f1_score(target,y_class_predict,zero_division=1,average='macro')
        train_recall += recall_score(target,y_class_predict,zero_division=1,average='macro')
        train_precision += precision_score(target,y_class_predict,zero_division=1,average='macro')
        train_loss += loss.item()

    model.eval()
    #with torch.no_grad():
    for batch_num, (input_img, target) in enumerate(validation_loader):


            output = model(input_img.to(device))
            target = target.to(device)
            loss = loss_fn(output, target)

            y_class_predict = torch.argmax(output, dim=1)
            y_class_predict = (y_class_predict.cpu().detach().numpy())

            target = target.cpu().detach().numpy()

            val_acc  += accuracy_score(target,y_class_predict)
            val_f1 += f1_score(target,y_class_predict,zero_division=1,average='macro')
            val_recall += recall_score(target,y_class_predict,zero_division=1,average='macro')
            val_precision += precision_score(target,y_class_predict,zero_division=1,average='macro')

            val_loss += loss.item()

            #print(f"Pèrdua entrenament batch validacio: {batch_num} epoch: {epoch+1}  val_loss: {val_loss.item()}")

            # RESULTATS
    train_loss /= len(train_loader)
    t_loss[epoch - 1] = train_loss

    train_acc /= len(train_loader)
    acc_t[epoch - 1] = train_acc

    train_f1 /= len(train_loader)
    f1_t[epoch - 1] = train_f1

    train_recall /= len(train_loader)
    recall_t[epoch - 1] = train_recall

    train_precision /= len(train_loader)
    precision_t[epoch-1] = train_precision

    print(f"Pèrdua entrenament epoch: {epoch}  train_loss: {train_loss}")
    print(f"Accuracy train epoch: {epoch}  train_acc: {train_acc}")
    print(f"F1 train epoch: {epoch}  train_f1: {train_f1}")
    print(f"Recall train epoch: {epoch}  train_recall: {train_recall}")
    print(f"Precision train epoch: {epoch}  train_recall: {train_precision}")

    val_loss /= len(validation_loader)
    v_loss[epoch - 1] = val_loss

    val_acc /= len(validation_loader)
    acc_v[epoch - 1] = val_acc

    val_f1 /= len(validation_loader)
    f1_v[epoch - 1] = val_f1

    val_recall /= len(validation_loader)
    recall_v[epoch - 1] = val_recall

    val_precision /= len(validation_loader)
    precision_v[epoch-1] = val_precision

    print()
    print()
    print(f"Pèrdua validació epoch: {epoch}  val_loss: {val_loss}")
    print(f"Accuracy val epoch: {epoch}  val_acc: {val_acc}")
    print(f"F1 val epoch: {epoch}  val_f1: {val_f1}")
    print(f"Recall val epoch: {epoch}  val_reall: {val_recall}")
    print(f"Precision val epoch: {epoch}  val_reall: {val_precision}")

  0%|          | 0/95 [00:00<?, ?it/s]

Batches (Època 1): 0it [00:00, ?it/s]

Pèrdua entrenament epoch: 1  train_loss: 4.4637552385744845
Accuracy train epoch: 1  train_acc: 0.029211956521739132
F1 train epoch: 1  train_f1: 0.0028901715626316565
Recall train epoch: 1  train_recall: 0.03819026553087201
Precision train epoch: 1  train_recall: 0.9603328036602963


Pèrdua validació epoch: 1  val_loss: 4.287948290506999
Accuracy val epoch: 1  val_acc: 0.07395833333333333
F1 val epoch: 1  val_f1: 0.013059289037813393
Recall val epoch: 1  val_reall: 0.08730235500813249
Precision val epoch: 1  val_reall: 0.8840508131106325


Batches (Època 2): 0it [00:00, ?it/s]

Pèrdua entrenament epoch: 2  train_loss: 4.050104688561481
Accuracy train epoch: 2  train_acc: 0.1479405034324943
F1 train epoch: 2  train_f1: 0.04022559960526865
Recall train epoch: 2  train_recall: 0.17031877592338499
Precision train epoch: 2  train_recall: 0.8120931487262313


Pèrdua validació epoch: 2  val_loss: 3.8089978377024334
Accuracy val epoch: 2  val_acc: 0.19895833333333332
F1 val epoch: 2  val_f1: 0.06249932279388571
Recall val epoch: 2  val_reall: 0.23231993470351497
Precision val epoch: 2  val_reall: 0.7298401062237476


...

Pèrdua entrenament epoch: 20  train_loss: 0.34653532161660816
Accuracy train epoch: 20  train_acc: 0.8570509153318078
F1 train epoch: 20  train_f1: 0.7593979365577274
Recall train epoch: 20  train_recall: 0.8861502259941575
Precision train epoch: 20  train_recall: 0.8464562103373247


Pèrdua validació epoch: 20  val_loss: 6.96960121790568
Accuracy val epoch: 20  val_acc: 0.4541118421052631
F1 val epoch: 20  val_f1: 0.2431796105429802
Recall val epoch: 20  val_reall: 0.5437571657241985
Precision val epoch: 20  val_reall: 0.566862006723276


Batches (Època 21): 0it [00:00, ?it/s]

Pèrdua entrenament epoch: 21  train_loss: 0.25344359396916366
Accuracy train epoch: 21  train_acc: 0.88245852402746
F1 train epoch: 21  train_f1: 0.7996909458896495
Recall train epoch: 21  train_recall: 0.9082528024958418
Precision train epoch: 21  train_recall: 0.8691161490131912


Pèrdua validació epoch: 21  val_loss: 6.82495485941569
Accuracy val epoch: 21  val_acc: 0.4496710526315789
F1 val epoch: 21  val_f1: 0.2363628932055008
Recall val epoch: 21  val_reall: 0.5401256816862211
Precision val epoch: 21  val_reall: 0.5379461196048315


Batches (Època 22): 0it [00:00, ?it/s]

KeyboardInterrupt: 